In [ ]:
from IPython.core.display import HTML
css_file = './custom.css'
HTML(open(css_file, "r").read())

###### Content provided under a Creative Commons Attribution license, CC-BY 4.0; code under MIT License. (c)2015-2023 [David I. Ketcheson](http://davidketcheson.info)

##### Version 0.4 - March 2023

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
font = {'size'   : 15}
matplotlib.rc('font', **font)

fft = np.fft.fft
ifft = np.fft.ifft

In [ ]:
def plot_solution(frames, uuhat, x, tt, xi):
    fig = plt.figure(figsize=(12,8))
    axes = fig.add_subplot(211)
    axes2 = fig.add_subplot(212)
    line, = axes.plot(x,frames[0],lw=3)
    line2, = axes2.semilogy(xi,np.abs(np.fft.fft(frames[0])))
    xi_max = np.max(np.abs(xi))
    axes2.semilogy([xi_max/2.,xi_max/2.],[1.e-6,4e8],'--r')
    axes2.semilogy([-xi_max/2.,-xi_max/2.],[1.e-8,4e10],'--r')
    axes.set_xlabel(r'$x$',fontsize=30)
    axes2.set_xlabel(r'$\xi$',fontsize=30)
    plt.tight_layout()
    plt.close()

    def plot_frame(i):
        line.set_data(x,frames[i])
        power_spectrum = np.abs(uuhat[i])**2
        line2.set_data(np.sort(xi),power_spectrum[np.argsort(xi)])
        axes.set_title('t= %.2e' % tt[i])
        axes.set_xlim((-np.pi,np.pi))
        axes.set_ylim((-200,3000))

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                              frames=len(frames), interval=100,
                                              repeat=False)
    return HTML(anim.to_jshtml())

# ImEx methods

Next we consider the use of ImEx additive Runge-Kutta methods.  In these methods, $f$ is handled explicitly while $g$ is handled implicitly.  An ImEx RK method takes the form

\begin{align} \label{ark}
    y_i & = u^n + h  \sum_{j=1}^{i} a_{ij} L(y_j) + h \sum_{j=1}^{i-1}\hat{a}_{ij} f(y_j) \\
    u^{n+1} & = u^n + h \sum_{j=1}^s \left(b_j L(y_j) + \hat{b}_j f(y_j)\right).
\end{align}

The method can be seen as a combination of a diagonally-implicit RK method with coefficients $(A,b)$ and an explicit RK method with coefficients $(\hat{A},\hat{b})$.

The idea is to benefit from treating the stiff linear term $L$ implicitly and the non-stiff nonlinear term $f$ explicitly.  Stability of additive Runge-Kutta methods is complicated and a subject of ongoing research, but roughly speaking one hopes that if the implicit part of the method is A-stable then the overall method will be stable if the time step is chosen so that the explicit integrator applied to $u_t=f(u)$ would be stable.

The coefficients of an additive Runge-Kutta method must satisfy additional order conditions beyond those required for the component methods.

The implementation here uses a scheme developed by [Higueras et. al.](https://www.sciencedirect.com/science/article/pii/S0377042714002477) (see Eqn. (17) in that paper).  It has the following useful properties:

- Second-order accuracy
- Some imaginary axis stability for the explicit method
- L-stability for the implicit method

The method coefficients are:

\begin{align}
\begin{array}{c|ccc}
0 & 0 & 0 & 0 \\
\frac{5}{6} & \frac{5}{6} & 0 & 0 \\
\frac{11}{12} & \frac{11}{24} & \frac{11}{24} & 0 \\
\hline
 & \frac{24}{55} & \frac{1}{5} & \frac{4}{11}
\end{array}
& &
\begin{array}{c|ccc}
\frac{2}{11} & \frac{2}{11} & 0 & 0 \\
\frac{289}{462} & \frac{205}{462} & \frac{2}{11} & 0 \\
\frac{751}{924} & \frac{2033}{4620} & \frac{21}{110} & \frac{2}{11} \\
\hline
 & \frac{24}{55} & \frac{1}{5} & \frac{4}{11}
\end{array}
\end{align}

The explicit part of this method includes the interval $[-1.2i,1.2i]$ in its stability interval, so we incorporate this factor into the time step size.

## Efficient implementation

In the implementation of pseudospectral methods, it's important to use the FFT and inverse FFT rather than naively performing dense matrix multiplications and solves.  For a pseudospectral discretization, we have

$$
    L = F^{-1} D F,
$$
where $D$ is a diagonal matrix and $F, F^{-1}$ are the discrete Fourier transform and its inverse.

one stage of an ImEx RK method then takes the form
$$
    y_i = u^n + h  \sum_{j=1}^{i} a_{ij} F^{-1} D F y_j + h \sum_{j=1}^{i-1}\hat{a}_{ij} f(y_j).
$$

Solving this for $y_i$ gives
$$
    y_i = (I-a_{ii} h F^{-1} D F)^{-1} \left(u^n + h  \sum_{j=1}^{i-1} a_{ij} F^{-1} D F y_j + h \sum_{j=1}^{i-1}\hat{a}_{ij} f(y_j)\right).
$$

We can write

$$
    (I-a_{ii} h F^{-1} D F)^{-1} = F^{-1}(I-a_{ii} h D)^{-1} F.
$$

Note that $I-a_{ii} h D$ is a diagonal matrix, so its inverse is trivial to compute.  Then the stage can be implemented in the form

$$
    y_i = F^{-1} (I-a_{ii} h D)^{-1} F \left(u^n + h  \sum_{j=1}^{i-1} a_{ij} F^{-1} D F y_j + h \sum_{j=1}^{i-1}\hat{a}_{ij} f(y_j)\right).
$$

In [ ]:
# Higueras (17)
A    = np.array([[0,0,0],[5/6.,0,0],[11/24,11/24,0]])
Ahat = np.array([[2./11,0,0],[205/462.,2./11,0],[2033/4620,21/110,2/11]])
b = np.array([24/55.,1./5,4./11])
bhat = b

def rhs_f(u, xi, filtr):
    # Evaluate only the non-stiff nonlinear term
    uhat = np.fft.fft(u)
    return -u*np.real(np.fft.ifft(1j*xi*uhat*filtr))

def solve_KdV_ImEx2(m,dt,use_filter=True):
    L = 2*np.pi
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L

    u = 1500*np.exp(-10*(x+2)**2)
    tmax = 0.05

    uhat2 = np.abs(np.fft.fft(u))

    num_plots = 50
    nplt = np.floor((tmax/num_plots)/dt)
    nmax = int(round(tmax/dt))

    frames = [u.copy()]
    tt = [0]
    uuhat = [uhat2]

    filtr = np.ones_like(xi)
    
    if use_filter:
        xi_max = np.max(np.abs(xi))
        filtr[np.where(np.abs(xi)>xi_max*2./3)] = 0.

    # Assumes all diagonal entries of Ahat are equal:
    Lfactor = 1/(1-Ahat[0,0]*dt*1j*xi**3)
    D = 1j*xi**3
    
    for n in range(1,nmax+1):
        y1 = ifft(Lfactor*fft(u))
        Ly1 = ifft(D*fft(y1))
        fy1 = rhs_f(y1,xi,filtr)
        y2rhs = u + dt*(A[1,0]*fy1+Ahat[1,0]*Ly1)
        y2 = ifft(Lfactor*fft(y2rhs))
        Ly2 = ifft(D*fft(y2))
        fy2 = rhs_f(y2,xi,filtr)
        y3rhs = u + dt*(A[2,0]*fy1+A[2,1]*fy2 + Ahat[2,0]*Ly1 + \
                        Ahat[2,1]*Ly2)
        y3 = ifft(Lfactor*fft(y3rhs))
        Ly3 = ifft(D*fft(y3))
        fy3 = rhs_f(y3,xi,filtr)
        u_new = u + dt*np.real(b[0]*(fy1+Ly1) + b[1]*(fy2+Ly2) + \
                               b[2]*(fy3+Ly3))
            
        u = u_new.copy()
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            frames.append(u.copy())
            tt.append(t)
            uhat2 = np.abs(np.fft.fft(u))
            uuhat.append(uhat2)
    return frames, uuhat, x, tt, xi

In [ ]:
m = 1024
umax = 3000
dt = 1.2/(m/2)/umax
use_filter=False

frames, uuhat, x, tt, xi = solve_KdV_ImEx2(m,dt,use_filter)

In [ ]:
plot_solution(frames, uuhat, x, tt, xi)

We note that this method appears to be stable without filtering, as long as our spatial grid is fine enough to resolve the solution.

# Concluding remarks
In the [comparison by Kassam & Trefethen](https://epubs.siam.org/doi/epdf/10.1137/S1064827502410633), ImEx multistep methods did not perform well.  In fact, they did not work at all for the KdV equation in particular.  Here we see that an ImEx Runge-Kutta method with appropriately-designed properties seems to work extremely well for the KdV equation.  In fact, out of all the efficient time-stepping methods we will study, this is the only one that does not seem to require filtering.